## 로컬 환경에서 PDF 검색하기 2단계 step1
- PDF 문서를 로드하고 한국어 임베딩 모델을 사용하여 임베딩 데이터 생성
- 임베딩 데이터를 csv 파일로 만들어 저장하기 

### - 사용한 임베딩 모델 jhgan/ko-sroberta-multitask


In [6]:
# 라이브러리 설치 (필요한 경우 실행)
%pip install -U langchain langchain_core sentence-transformers faiss-cpu pymupdf

Note: you may need to restart the kernel to use updated packages.


In [9]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  
from sentence_transformers import SentenceTransformer

# Step 1: 문서 로드
pdf_file_path = "data/"
pdf_file_name = "AI기반_인파분석플랫폼구축_제안서"

loader = PyMuPDFLoader(pdf_file_path + pdf_file_name + ".pdf") 
docs = loader.load()

# Step 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [10]:
# Step 3: SentenceTransformer 모델을 LangChain의 Embeddings 클래스로 감싸기
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        """문서 리스트를 벡터로 변환"""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """검색 쿼리를 벡터로 변환"""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


# Step 4: 모델 로드 및 FAISS 인덱스 생성
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")
# faiss_index = FAISS.from_documents(split_documents, embedding_model)

In [12]:
import csv
import pandas as pd
import os

# Step 5: 문서 임베딩 및 CSV 저장
# 원본 문서와 임베딩 데이터를 CSV 에 함께 저장함
# 원본 문서를 저장하는 이유 : 검색 결과를 보여줘야 하기 때문. 
def save_embeddings_to_csv(documents, embedding_model, filename=pdf_file_name+".csv", file_path="./csv/"):
    # 경로가 존재하지 않은 경우 디렉토리 생성
    os.makedirs(file_path, exist_ok=True)
    full_path = os.path.join(file_path, filename)

    # 문서 임베딩 수행
    embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])
    
    # CSV 저장
    with open(full_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["document", "embedding"])
        
        for doc, embedding in zip(documents, embeddings):
            writer.writerow([doc.page_content, embedding])
    
    print(f"임베딩 데이터가 {full_path} 파일에 저장되었습니다.")

# 함수 실행
documents = split_documents  # FAISS에 넣은 문서 리스트 사용
save_embeddings_to_csv(documents, embedding_model)

임베딩 데이터가 ./csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.
